<a href="https://colab.research.google.com/github/BrotherKim/KAIST_CODE/blob/master/SEP531/3_KoBERT_finetuend_model_f1score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
ROOT_DIR = '/content/drive/MyDrive/KAIST/SEP531/kor_ks_labeled'
TRAIN_DIR = '%s/train' % ROOT_DIR
VALID_DIR = '%s/valid' % ROOT_DIR

NUM_TRAIN = 171
NUM_VALID = 58

SAVEPOINT_PATH = '/content/drive/MyDrive/KAIST/SEP531/KoBERT_emotion_finetuned_data_added.pt'
#SAVEPOINT_PATH = '/content/drive/MyDrive/KAIST/SEP531/KoBERT_emotion_finetuned.pt'

In [2]:
!ls -l /content/drive/MyDrive/KAIST/SEP531

total 741149
-rw------- 1 root root   1894595 Nov  8 06:21 4차년도.csv
-rw------- 1 root root   2142130 Dec  4 08:42 4_utf8.csv
-rw------- 1 root root   2167298 Dec  4 09:52 4차년도_utf8.csv
-rw------- 1 root root   2642158 Nov  8 06:21 5차년도_2차.csv
-rw------- 1 root root   2976104 Dec  4 08:41 5_2_utf8.csv
-rw------- 1 root root   2976104 Dec  4 09:52 5차년도_2차_utf8.csv
-rw------- 1 root root   1433551 Nov  8 06:21 5차년도.csv
-rw------- 1 root root   1659315 Dec  4 08:41 5_utf8.csv
-rw------- 1 root root   1678755 Dec  4 09:52 5차년도_utf8.csv
drwx------ 2 root root      4096 Nov 23 14:55 hw3_cp
-rw------- 1 root root 368848687 Nov  8 07:19 KoBERT_emotion_finetuned_data_added.pt
-rw------- 1 root root 368848687 Nov  6 07:19 KoBERT_emotion_finetuned.pt
drwx------ 2 root root      4096 Nov 21 00:02 kor_cc_labeled
drwx------ 2 root root      4096 Nov 21 00:02 kor_jj_labeled
drwx------ 2 root root      4096 Nov 21 00:02 kor_jr_labeled
drwx------ 2 root root      4096 Nov  6 08

#KoBERT 다운로드(사용)#

In [3]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-wpwfasvg
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-wpwfasvg


In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [5]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [6]:
#GPU 사용
device = torch.device("cuda:0")

In [7]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


#실행 환경(사용)#

- Python >= 3.6
- PyTorch >= 1.70
- Transformers = 3.0.2
- Colab
- batch size = 64 (convertable)
- epochs = 10 (convertable)

In [8]:
!pip install -U torchtext==0.6.0
!pip install transformers

In [9]:
#!pip install mxnet
#!pip install gluonnlp pandas tqdm
#!pip install sentencepiece
#!pip install transformers==3.0.2
#!pip install torch

# Libraries

import matplotlib.pyplot as plt
import pandas as pd
import torch

# Preliminaries

from torchtext.data import Field, TabularDataset, BucketIterator, Iterator

# Models

import torch.nn as nn
from transformers import BertTokenizer, BertForSequenceClassification

# Training

import torch.optim as optim

# Evaluation

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

In [10]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#KoBERT 입력 데이터로 변환(사용)#

In [11]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [12]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

#추출한 데이터에 감정 레이블 추가(사용)#

In [13]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

새로운 문장을 테스트 할 때, 입력되는 문장을 KoBERT의 입력 형식으로 바꿔주는 코드를 작성해주어야 한다. 아래 코드를 작성하여 토큰화, 패딩, 텐서를 바꿔주고 예측을 하는 'predict' 함수를 만들어 주었다.

In [14]:
model1 = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
model1.load_state_dict(torch.load(SAVEPOINT_PATH), strict=False)
model1.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [34]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

dic = {0:'공포', 1:'놀람', 2:'분노', 3:'슬픔', 4:'중립', 5:'행복', 6:'혐오'}
dic = {0:'fear', 1:'surprise', 2:'anger', 3:'sadness', 4:'neutral', 5:'happiness', 6:'disgust'}
r_dic = {'fear':0, 'surprise':1, 'anger':2, 'sadness':3, 'neutral':4, 'happiness':5, 'disgust':6}
def getEmotion(out):
  idx_eval=[]
  val_eval=[]
  for i in out:
    logits=i
    logits = logits.detach().cpu().numpy()
    idx = np.argmax(logits)
    idx_eval.append(idx)
    val_eval.append(dic.get(idx))
    return idx_eval[0], val_eval[0]


def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model1.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model1(token_ids, valid_length, segment_ids)

        return getEmotion(out)


        # file wrting

using cached model


#Eval

In [42]:
r_dic.get('sadness')

3

In [53]:
import csv

y = []
p = []
def Eval(filename):
  with open(filename, 'r') as f:
    csv_data = csv.reader(f, delimiter=',')
    next(csv_data)
    for line in csv_data:
      text = line[1]
      label = line[2]
      label_idx = r_dic.get(label)
      idx, val = predict(text)
      #print('(%s/%s) %s\n' % (val, label, text))
      print('(%c)(%d/%d) %s\n' % ('O' if (idx == label_idx) else 'X', idx, label_idx, text))
      p.append(idx)
      y.append(label_idx)
  #print('>> Generated file %s/%s.answer\n' % (path, jsonfilename))

  #with open('%s/%s.data' % (path, jsonfilename), 'w') as f:
  #  for u in ut:
  #    f.write('%s\t%s\n' % (u['standard_form'], u['dialect_form']))


In [54]:
Eval(train_csv)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


(X)(4/2) 어, 청소 니가 대신 해 줘!

(X)(3/2) 둘 다 청소 하기 싫어. 귀찮아.

(O)(2/2) 둘 다 하기 싫어서 화내.

(X)(3/2) 그럼 방세는 어떡해.

(O)(3/3) 권태긴줄 알았는데 다른 사람이 생겼나보더라고.

(X)(4/3) 그냥 걷고 있어.

(O)(2/2) 어. 고등학교 동창인데 이렇게 더럽게 쓸줄 몰랐어.

(X)(4/3) 처음 학원에서 만났다가 서로 좋아해서 사귀게 되었지.

(X)(4/3) 내가 애정 표현을 잘 못해서 자주 싸우긴 했어.

(O)(3/3) 오늘 헤어졌어.

(O)(2/2) 룸메이트와 너무 자주 싸우게 돼.

(X)(3/2) 그러고 싶은데 보증금 때문에 그럴 수가 없어.

(O)(3/3) 이 회사가 이번 시즌 마지막 회사였어.

(X)(0/3) 부모님한테 아직 말 안했는데 말하기가 두려워.

(O)(3/3) 아니. 입맛도 없어.

(X)(2/3) 응. 혼 났지.

(O)(3/3) 그럴 시간도 없다.

(O)(3/3) 부모님도 다 슬퍼하셔.

(X)(5/3) 그래, 고마워.

(O)(2/2) 해봤는데, 전화를 안 받네.

(X)(4/2) 맨날 그래 얘는.

(X)(3/2) 어. 크게 다치진 않았는데.

(X)(4/2) 일주일에 다섯번은 먹는다니까?

(O)(3/3) 그저께 아파해서 병원을 갔는데 괜찮아지나 싶더니 결국엔 이렇게 됐네.

(O)(3/3) 다들 마음이 안좋지. 집에 들어가기도 싫네, 나는.

(O)(3/3) 응. 그래봐야지. 해피도 오래 살긴 했어.

(X)(3/2) 그렇지. 뭐 생명까지는 괜찮은데. 우리 마음에 지장이 있네.

(X)(3/2) 지난 번에도 드시고 다친 적 있었거든, 얼굴? 이번에 또 그런 거야.

(X)(3/2) 다들 지쳤어. 나도 지쳤구, 이제.

(O)(3/3) 어제 밤 새서 작성한 기획안 다시 해야되네.

(X)(4/3) 그렇지. 아픈데도 했더니, 참, 근데 맞는 말이라서.

(O)(3/3) 응. 마음 좀 추스려야지. 근데 내가 잘하는게 없는 거 같네.

(X)(4

Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 232, in _feed
    close()
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 232, in _feed
    close()
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
(O)(3/3) 실수한 거 만회해야돼.

(O)(3/3) 일찍 퇴근할 수가 없어.

(O)(3/3) 나이가 좀 있어서 그런 거 같애.

(O)(3/3) 아직은 힘들어하시네.

(O)(3/3) 좋은데로 보내드려야지.

(O)(3/3) 3년 동안 사귄 남자친구랑 결국 헤어졌어.

(O)(2/2) 약속 시간이 지났는데, 친구가 안 오네.

(O)(2/2) 연락을 하는데, 지금 온다고 하는데, 속상하네.

(O)(2/2) 말을 했는데 잘 지켜지지가 않네.

(X)(3/2) 어제 우리 아빠가 또 다치셨어.

(X)(3/2) 술 마시고 넘어지셨어.

(X)(3/2) 지장은 없는데 조금 힘들어하시네.

(X)(4/3) 3년 전에 친구 소개로 만났어.

(O)(0/0) 어제 저녁에 진짜 무서웠어. 너 느꼈어?

(O)(2/2) 이야기를 하는데도 잘 안됐네.

(X)(3/0) 응. 나는 괜찮은데 같이 있던 친구는 허둥대다가 발목을 좀 삐었어.

(O)(2/2) 퇴근한 지 얼마나 됐다고, 또 오라고 하는 거야.

(O)(3/3) 헤어진지 두 달 정도 됐는데 아직도 적응이 안돼. 생각만 하면 눈물이 나고 가슴이 아파.

(O)(2/2) 그래두 다녀야지.

(X)(3/2) 요즘 세상엔 갈데가 없어.

(X)(4/2) 응. 회사 동료. 동생도 있고.

(X)(4/2) 그래, 알겠어.

(O)(0/0) 어 정말 무섭더라고 어떻게 해야될지 알고 있었다고 생각했거든? 근데 막상 상황이 되니까 발만 동동 구르고 있게 되더라고. 다른 사람들도 똑같앴어. 정말 무서워하더라니까?

(O)(6/6) 화장실 세면대랑 바닥에 토가 그대로 있어. 너무 더러워.

(O)(0/0) 아직도 진정이 안되고 있어.

(X)(4/0) 조금 다친 거 같아.

(X)(4/6) 속이 안 좋은데 따뜻한 물을 마시면 좀 나아질 거 같아.

(O)(6/6) 응. 아직도 그대로야.

(X)(4/6) 그래야 되겠어. 다른 층을 한 번 가봐야 되겠어.

(X)(4/

#F1 score

In [67]:
with open('pred.csv', 'w') as f:
  for d in p:
    f.write('%d,' % (d))
len(p)

14606

In [66]:
with open('label.csv', 'w') as f:
  for d in y:
    f.write('%d,' % (d))
len(y)

14606

In [58]:
p

[4,
 3,
 2,
 3,
 3,
 4,
 2,
 4,
 4,
 3,
 2,
 3,
 3,
 0,
 3,
 2,
 3,
 3,
 5,
 2,
 4,
 3,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 3,
 4,
 3,
 2,
 4,
 3,
 3,
 0,
 3,
 4,
 3,
 2,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 3,
 4,
 3,
 4,
 3,
 3,
 2,
 1,
 4,
 3,
 3,
 2,
 2,
 2,
 6,
 4,
 3,
 3,
 3,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 4,
 3,
 3,
 4,
 3,
 3,
 3,
 2,
 2,
 2,
 4,
 3,
 2,
 2,
 3,
 3,
 6,
 6,
 4,
 2,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 2,
 2,
 0,
 6,
 6,
 4,
 2,
 2,
 3,
 3,
 2,
 3,
 3,
 2,
 3,
 2,
 4,
 3,
 4,
 3,
 3,
 3,
 4,
 2,
 2,
 4,
 2,
 2,
 4,
 3,
 4,
 3,
 4,
 4,
 4,
 2,
 2,
 2,
 2,
 3,
 3,
 4,
 2,
 2,
 2,
 3,
 2,
 4,
 2,
 5,
 0,
 0,
 4,
 4,
 4,
 6,
 4,
 6,
 3,
 4,
 4,
 3,
 2,
 2,
 2,
 2,
 2,
 1,
 0,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 6,
 6,
 1,
 6,
 5,
 0,
 4,
 3,
 2,
 3,
 3,
 3,
 4,
 3,
 3,
 3,
 3,
 3,
 4,
 3,
 3,
 3,
 3,
 2,
 2,
 2,
 3,
 4,
 3,
 3,
 3,
 6,
 4,
 2,
 4,
 2,
 4,
 4,
 3,
 3,
 3,
 6,
 6,
 6,
 2,
 3,
 3,
 3,
 0,
 0,
 3,
 4,
 3,
 3,
 3,
 3,
 2,
 4,
 3,
 2,
 2,
 2,
 4,
 3,
 3,
 3,
 2,
 3,
 2,


In [71]:
print(metrics.precision_score(y, p, average=None)) #
print(metrics.precision_score(y, p, average=None).mean()) #
print(metrics.precision_score(y, p, average='macro')) #1.0
print(metrics.precision_score(y, p, average='micro')) #1.0

[0.93203883 0.         0.88199276 0.75765625 0.00183688 0.
 0.87878788]
0.4931875160128841
0.4931875160128841
0.6429549500205395


In [74]:
import sklearn.metrics as metrics

# sklearn 을 이용하면 전부 계산해준다.
print('accuracy', metrics.accuracy_score(y,p) )
print('precision1', metrics.precision_score(y, p, average=None)) #
print('precision2', metrics.precision_score(y, p, average=None).mean()) #
print('precision3', metrics.precision_score(y, p, average='macro')) #1.0
print('precision4', metrics.precision_score(y, p, average='micro')) #1.0
print('recall1', metrics.recall_score(y, p, average=None)) #
print('recall2', metrics.recall_score(y, p, average=None).mean()) #
print('recall3', metrics.recall_score(y, p, average='macro')) #1.0
print('recall4', metrics.recall_score(y, p, average='micro')) #1.0
print('f1_score1', metrics.f1_score(y, p, average=None)) #
print('f1_score2', metrics.f1_score(y, p, average=None).mean()) #
print('f1_score3', metrics.f1_score(y, p, average='macro')) #1.0
print('f1_score4', metrics.f1_score(y, p, average='micro')) #1.0

print(metrics.classification_report(y,p))
print(metrics.confusion_matrix(y,p))

accuracy 0.6429549500205395
precision1 [0.93203883 0.         0.88199276 0.75765625 0.00183688 0.
 0.87878788]
precision2 0.4931875160128841
precision3 0.4931875160128841
precision4 0.6429549500205395
recall1 [0.42504744 0.         0.60790332 0.77066116 1.         0.
 0.45940594]
recall2 0.4661454077965049
recall3 0.4661454077965049
recall4 0.6429549500205395
f1_score1 [0.58384014 0.         0.71973654 0.76410337 0.00366703 0.
 0.60338101]
f1_score2 0.38210401459325277
f1_score3 0.38210401459325277
f1_score4 0.6429549500205395
              precision    recall  f1-score   support

           0       0.93      0.43      0.58      1581
           1       0.00      0.00      0.00         0
           2       0.88      0.61      0.72      5213
           3       0.76      0.77      0.76      6292
           4       0.00      1.00      0.00         5
           5       0.00      0.00      0.00         0
           6       0.88      0.46      0.60      1515

    accuracy                     

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill